In [22]:
# import libraries

import openai
import os
import glob
from dotenv import load_dotenv, find_dotenv
from langchain.evaluation import load_evaluator
from langchain.llms import OpenAI
import gradio as gr

import pandas as pd
import json

In [23]:
import warnings
warnings.filterwarnings("ignore")

In [39]:
# load and read data.json

with open('data.json') as f:
  data = json.load(f)

In [41]:
def eval_input(source,essay_type,essay):
    source,essay_type,essay = source,essay_type,essay

demo = gr.Interface(
    eval_input,
    ["text","text","text"],
    None,
)
demo.launch()

Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


In [27]:
# load the env.txt that contains the OPENAI key

load_dotenv('env.txt')
openai_api_key = os.getenv('OPENAI_API_KEY')

In [28]:
llm = OpenAI(
    openai_api_key=openai_api_key,
    model ='gpt-3.5-turbo-instruct'
)

In [29]:
narrative_custom_criterion = {
    "grammar": "Does the output follow proper grammar, spelling, and pronunciation rules?",
    "coherence": "Does the output have a coherent flow of ideas?",
    "vocabulary":"Does the output use a robust dictionary of vocabulary?",
    "relevance":"Does the output fully address the task?",
    "description":"Does the output include sensory words?",
    "characters":"Does the output capture the personality or emotions of the characters in the story?",
    "openings": "Does the output have an engaging opening?"
}

In [30]:
argumentative_custom_criterion = {
    "grammar": "Does the output follow proper grammar, spelling, and pronunciation rules?",
    "coherence": "Does the output have a coherent flow of ideas?",
    "vocabulary":"Does the output use a robust dictionary of vocabulary?",
    "relevance":"Does the output fully address the task?",
    "attention":"Does the output capture your attention?",
    
}

In [31]:
evaluator = load_evaluator("pairwise_string", criteria="custom_criterion", llm=llm)

This chain was only tested with GPT-4. Performance may be significantly worse with other models.


In [32]:
print(evaluator)

prompt=ChatPromptTemplate(input_variables=['input', 'prediction', 'prediction_b'], partial_variables={'reference': '', 'criteria': 'For this evaluation, you should primarily consider the following criteria:\ncustom_criterion'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user question displayed below. You should choose the assistant that follows the user\'s instructions and answers \the user\'s question better. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of their responses. Begin your evaluation by comparing the two responses and provide a short explanation. Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. Do not allow the length of the responses to influence your evaluation. D

In [33]:
def eval(source,essay_type,essay):
    source,essay_type,essay = source,essay_type,essay

In [42]:
demo = gr.Interface(
    eval,
    ["text","text","text"],
    "text",
    title = f'O level Essay Grader'
)
demo.launch()  

Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.
